<a href="https://colab.research.google.com/github/adamjweintraut/W209_Final_Project/blob/main/Week_8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Networks in Altair

In [2]:
from IPython.display import HTML
HTML("""<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Draw interactive <a href="https://twitter.com/hashtag/NetworkX?src=hash&amp;ref_src=twsrc%5Etfw">#NetworkX</a> graphs with <a href="https://twitter.com/hashtag/Altair?src=hash&amp;ref_src=twsrc%5Etfw">#Altair</a>.<br><br>nx_altair now supports most of the drawing features in NetworkX. Check it out!<a href="https://t.co/nS4g21iQrR">https://t.co/nS4g21iQrR</a> <a href="https://t.co/EDHeAE5rGt">pic.twitter.com/EDHeAE5rGt</a></p>&mdash; Zach Sailer (@zrsailer) <a href="https://twitter.com/zrsailer/status/995720750899249152?ref_src=twsrc%5Etfw">May 13, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>""")

Code is on github: [https://github.com/Zsailer/nx_altair](https://github.com/Zsailer/nx_altair)

In [3]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [142]:
%matplotlib inline
import pandas as pd
import numpy as np
import networkx as nx
import altair as alt
import nx_altair as nxa
from itertools import combinations


# Upload tweets.json

In [ ]:
from google.colab import files
files.upload()

# Parse and Set up for Network

In [220]:
tweets = pd.read_json("tweets.json")
# parse
tweets['hashtag_obj_list'] = tweets['entities'].str['hashtags']
tweets['hashtag_str_list'] = [[hashtag['text'] for hashtag in hashtag_list ] for hashtag_list in tweets['hashtag_obj_list'] ]

# this is our base.
tweet_hashtags = tweets.explode('hashtag_str_list').rename(columns={'hashtag_str_list': 'hashtag'}).dropna(axis=0, subset=['hashtag'])[['id', 'hashtag']]

# use this to create combinational EDGES
graph_tweets = pd.DataFrame(tweet_hashtags.groupby('id')['hashtag'].apply(list)).rename(columns = {'hashtag':'hashtags'}).reset_index()

# use this to get weight & name for nodes
hashtag_grouped = pd.DataFrame(tweet_hashtags.groupby('hashtag')['id'].count()).rename(columns = {'id':'num_tweets'}).reset_index()
# hashtag_grouped
# tweets.head()

# Create Nodes, Edges

In [309]:
G = nx.Graph()

nodes = hashtag_grouped['hashtag'].tolist()
G.add_nodes_from(nodes)

# Add attributes to each node.
for n in G.nodes():
    G.nodes[n]['num_tweets'] = hashtag_grouped.loc[hashtag_grouped['hashtag'] == n]['num_tweets'].values[0]
    G.nodes[n]['weight'] = (np.log(hashtag_grouped.loc[hashtag_grouped['hashtag'] == n]['num_tweets'].values[0]) + 1) * 10000
    G.nodes[n]['name'] = n

edges = []
for hashtag_array in graph_tweets['hashtags']:
  edges.extend(list(combinations(hashtag_array, 2)))


G.add_edges_from(edges)

# Add attributes to each node.

for e in G.edges():
    G.edges[(e[0],e[1])]['weight'] = 75

# G.nodes()
# hashtag_grouped.loc[hashtag_grouped['hashtag'] == '100daysofdataviz']['num_tweets'].values[0]


# Create Graph.

In [310]:
# can take a second.
pos = nx.spring_layout(G)

In [311]:
# from matplotlib import pyplot as plt

fig = plt.figure(1, figsize=(6, 6), dpi=100)

# Draw the graph using Altair (why no edges ..?)
nxa.draw_networkx(G=G,
                  pos=pos,
                  node_size=60,
                  node_color='num_tweets',
                  cmap='viridis',
                  width='weight',
                  node_tooltip=['num_tweets', 'name'])\
        .properties(width=900, height=900).interactive()


# show it
# viz



# ax


alt.LayerChart(...)

<Figure size 600x600 with 0 Axes>